In [3]:
#%pip install selenium
#%pip install webdriver_manager
#%pip install fake_useragent
from selenium import webdriver
import pdb

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import pandas as pd
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
import time
import random

In [10]:
# set the website to scrape
website = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
# set the exit boolean variable to False
last_page_reached = False

In [9]:
# Testing
# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# Open the IMDb website URL
driver.get(website)

# Perform various tests and assertions to probe the features
# Example:
# Test 1: Verify the presence of the movie titles on the page
movie_titles = driver.find_elements(By.XPATH, '//h3[@class="lister-item-header"]/a')
assert len(movie_titles) > 0

# Test 2: Click on a movie title and validate the details page
first_movie_title = movie_titles[0]
first_movie_title.click()


# Test 3: Navigate back to the search results page
driver.back()

# Add assertions to check if the search results page is loaded and contains the expected elements

# Close the browser
driver.quit()


In [4]:
# Initialize the UserAgent object to outsmart the website's bot detection
ua = UserAgent()
# Example list of proxies. 
proxies = ['ip1:port1', 'ip2:port2', 'ip3:port3']
# set the number of retry attempts
retry_attempts = 3
for proxy in proxies:
    # exit the loop if the last page is reached
    if last_page_reached:
        break
    # Set the User-Agent and Proxy
    user_agent = ua.random
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": proxy,
        "ftpProxy": proxy,
        "sslProxy": proxy,
        "proxyType": "MANUAL"
    }
    # Create a new instance of the Chrome driver options
    chrome_options = Options()
    # Set the User Agent and Proxy
    chrome_options.add_argument(f'user-agent={user_agent}')
    #run the browser in headless mode --> can help to avoid bot detection
    chrome_options.add_argument('--headless')

    for attempt in range(retry_attempts):
        try:
            # Create a new instance of the Chrome driver with the Chrome driver options
            driver = webdriver.Chrome(options=chrome_options)
            # Navigate to the website
            driver.get(website)
            # find the next page button
            next_page = driver.find_element(By.LINK_TEXT, 'Next »')
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Retrying...")
            pdb.post_mortem()
    else:
        print("Failed to retrieve page after multiple attempts. Moving to the next proxy.")
        continue
    # Initialize the lists to store the scraped data
    rang,film,jahr,fsk,dauer,genre,bewertung,regisseur,stars = [],[],[],[],[],[],[],[]

    # Loop until the last page is reached
    while next_page:
        # Get all the movie information on the page by XPATH
        movie_info = driver.find_elements(By.XPATH, '//div[@class="lister-item-content"]')
        for info in movie_info:
            # Split the text into lines and select the second line
            lines = info.text.split('\n')
            header = lines[0].split(' ')
            # Append the data to the lists
            rang.append(header[0])
            jahr.append(header[-1])
            film.append(header[1:-1])

            # Check for the length of the list to avoid errors
            if len(lines[1].split('|')) == 3:
                fsk.append(lines[1].split('|')[0])
                dauer.append(lines[1].split('|')[1])
                genre.append(lines[1].split('|')[2])
            elif len(lines[1].split('|')) == 2:
                fsk.append('NaN')
                dauer.append(lines[1].split('|')[0])
                genre.append(lines[1].split('|')[1])

            bewertung.append(lines[2].split(' ')[0])
            staff = lines[-2].split(" | ")
            # Iterating through the list and replacing 'Director:' with an empty string
            for i in range(len(staff)):
                staff[i] = staff[i].replace('Director: ', '')
                staff[i] = staff[i].replace('Stars: ', '')
            # Append the duration to the list
            regisseur.append(staff[0])
            stars.append(staff[-1])

        try:
            # Find the next page button and click on it
            next_page = driver.find_element(By.LINK_TEXT, 'Next »')
            next_page.click()
            # Sleep for a random time between 1 and 5 seconds to not trigger the bot detection
            time.sleep(random.randint(1, 5))
        except Exception as e:
            print("Last page reached")
            # Set the last_page_reached variable to True
            last_page_reached = True
            # Set the next_page variable to None
            next_page = None
            # Close the browser
            driver.quit()
            break

Last page reached


,rang,film,jahr,fsk,dauer,genre,bewertung,regisseur,stars
0,1.,"[Die, Verurteilten]",(1994),12,142 min,Drama,"9,3",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,2.,"[Der, Pate]",(1972),16,175 min,"Crime, Drama","9,2",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,3.,"[The, Dark, Knight]",(2008),16,152 min,"Action, Crime, Drama","9,0",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,4.,"[Schindlers, Liste]",(1993),12,195 min,"Biography, Drama, History","9,0",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."
4,5.,"[Der, Herr, der, Ringe:, Die, Rückkehr, des, K...",(2003),12,201 min,"Action, Adventure, Drama","9,0",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
...,...,...,...,...,...,...,...,...,...
995,996.,[Control],(2007),12,122 min,"Biography, Drama, Music","7,6",Anton Corbijn,"Sam Riley, Samantha Morton, Craig Parkinson, A..."
996,997.,"[Philomena:, Eine, Mutter, sucht, ihren, Sohn]",(2013),6,98 min,"Biography, Comedy, Drama","7,6",Stephen Frears,"Judi Dench, Steve Coogan, Sophie Kennedy Clark..."
997,998.,"[Shine, -, Der, Weg, ins, Licht]",(1996),12,105 min,"Biography, Drama, Music","7,6",Scott Hicks,"Geoffrey Rush, Armin Mueller-Stahl, Justin Bra..."
998,999.,"[Der, Unsichtbare]",(1933),16,71 min,"Horror, Sci-Fi","7,6",James Whale,"Claude Rains, Gloria Stuart, William Harrigan,..."


In [5]:
# create empty dataframe
df = pd.DataFrame()
# add columns to dataframe
df['rang'] = rang
df['film'] = film
df['jahr'] = jahr
df['fsk'] = fsk
df['dauer'] = dauer
df['genre'] = genre
df['bewertung'] = bewertung
df['regisseur'] = regisseur
df['stars'] = stars
df

,rang,film,jahr,fsk,dauer,genre,bewertung,regisseur,stars
0,1.,"[Die, Verurteilten]",(1994),12,142 min,Drama,"9,3",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,2.,"[Der, Pate]",(1972),16,175 min,"Crime, Drama","9,2",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,3.,"[The, Dark, Knight]",(2008),16,152 min,"Action, Crime, Drama","9,0",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,4.,"[Schindlers, Liste]",(1993),12,195 min,"Biography, Drama, History","9,0",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."
4,5.,"[Der, Herr, der, Ringe:, Die, Rückkehr, des, K...",(2003),12,201 min,"Action, Adventure, Drama","9,0",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
...,...,...,...,...,...,...,...,...,...
995,996.,[Control],(2007),12,122 min,"Biography, Drama, Music","7,6",Anton Corbijn,"Sam Riley, Samantha Morton, Craig Parkinson, A..."
996,997.,"[Philomena:, Eine, Mutter, sucht, ihren, Sohn]",(2013),6,98 min,"Biography, Comedy, Drama","7,6",Stephen Frears,"Judi Dench, Steve Coogan, Sophie Kennedy Clark..."
997,998.,"[Shine, -, Der, Weg, ins, Licht]",(1996),12,105 min,"Biography, Drama, Music","7,6",Scott Hicks,"Geoffrey Rush, Armin Mueller-Stahl, Justin Bra..."
998,999.,"[Der, Unsichtbare]",(1933),16,71 min,"Horror, Sci-Fi","7,6",James Whale,"Claude Rains, Gloria Stuart, William Harrigan,..."


In [ ]:
# export dataframe to csv
df.to_csv('../data/imdb_top_1000.csv', index=False)